In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from src.web_tasks.page_summary import WebsiteContentSummarizer
from rich.markdown import Markdown
from rich.console import Console
from src.config import OPENAIVARS

In [3]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [4]:
openai = OpenAI()
openai_model = OPENAIVARS.open_ai_model

### Surpress Logs

In [5]:
import logging

#Surpresses all logs below critical
logging.getLogger().setLevel(logging.CRITICAL)

# Restores logging
# logging.getLogger().setLevel(logging.DEBUG) 

### Prompts

In [6]:
system_prompt = (
    "You are a highly skilled **business analyst assistant**, specializing in analyzing website content and generating structured, detailed, and insightful business reports.\n\n"

    "**Your Responsibilities:**\n"
    "- Provide an in-depth **business analysis report** based strictly on the provided website content.\n"
    "- **Extract insights**, rather than simply summarizing the text.\n"
    "- Ignore **navigation elements** (menus, footers, sidebars, repetitive content).\n"
    "- Focus on **core business information** (services, products, mission, vision, customer engagement strategies, competitive positioning, upcoming events, etc.).\n"
    "- Highlight **unique business aspects** and **opportunities for growth**.\n"
    "- Identify **target demographics** who would benefit from the website’s offerings.\n\n"

    "**Response Format (STRICT Markdown Structure Required):**\n"
    "- The report **must be in Markdown** for readability.\n"
    "- Use **clear headings, bullet points, and structured content**.\n"
    "- If the website provides event details, include a structured table with event names, dates, and target audiences.\n"
    "- Incorporate **quotes or testimonials** from the website (if available) for added credibility.\n\n"

    "**Important Notes:**\n"
    "- Do not fabricate information—only analyze and interpret what is available.\n"
    "- Use structured Markdown formatting for better readability.\n"
)

user_prompt = (

    "**WEBSITE CONTENT PROVIDED BELOW:**\n\n"

    "**TASK:**\n"
    "Generate a **comprehensive and structured business analysis report** based strictly on the provided website content.\n\n"

    "**RESPONSE FORMAT REQUIREMENT:**\n"
    "- The report **must be written in Markdown format**.\n"
    "- Use **structured sections, tables (if applicable), and bullet points**.\n"
    "- Ensure clarity, depth, and proper segmentation of insights.\n\n"

    "**FINAL TOUCH:**\n"
    "Provide **a summary with key takeaways**, ensuring all insights are well-organized and actionable.\n\n"

    "**WEBSITE CONTENT STARTS BELOW:**"
)

links_filter_system_prompt = (
    "You are an **intelligent web assistant**, responsible for **analyzing a list of hyperlinks** extracted from a website.\n"
    "Your task is to filter and retain only the **most relevant and informative links**, ensuring that the extracted content aligns with the business analysis process.\n\n"

    "**Purpose & Context:**\n"
    "The selected links will be used by the **business analysis assistant** to generate a comprehensive business report.\n"
    "- `system_prompt`: Guides the assistant in summarizing a company's key offerings based on website content.\n"
    "- `user_prompt`: Instructs the assistant to generate an insightful business report focused on services, products, and competitive positioning.\n\n"

    "**Link Selection Criteria:**\n"
    "Only keep links that provide information crucial to understanding the **business, its offerings, and unique value proposition**"
    "This includes, but is NOT limited to. :\n"
    "- **About Us** (mission, history, brand identity)\n"
    "- **Products & Services** (detailed breakdown of offerings)\n"
    "- **Blog & News** (business insights, industry trends, company updates)\n"
    "- **Case Studies & Testimonials** (customer success stories, credibility indicators)\n"
    "- **Careers** (team culture, hiring insights, employer brand perception)\n\n"

    "**Exclude the following links:**\n"
    "- **Navigation or duplicate content** (e.g., menus, footers, repeated elements)\n"
    "- **Social media pages** (Facebook, Instagram, Twitter, LinkedIn, YouTube, etc.)\n"
    "- **Legal & policy pages** (Terms of Service, Privacy Policy, Cookie Policy)\n"
    "- **Contact pages that only contain an email form**\n"
    "- **External third-party websites**, unless they are integral to the business (e.g., official partner integrations)\n"
    "- **Generic FAQ pages that do not provide unique insights about the business**\n"
    "- **Sitemap, login pages, or any technical pages unrelated to business insights**\n\n"

    "**Response Format:**\n"
    "Provide a structured JSON response containing only the retained links with their type classification.\n"
    "Example output:\n"
    "{\n"
    "    'links': [\n"
    "        {'type': 'about', 'url': 'https://company.com/about'},\n"
    "        {'type': 'careers', 'url': 'https://company.com/careers'},\n"
    "        {'type': 'blog', 'url': 'https://company.com/blog'}\n"
    "    ]\n"
    "}"
)


links_filter_user_prompt = "Website: {url}\n\nHere is a list of links found on the website:\n" + "\n"

### Single Page Summary

In [7]:
summariser = WebsiteContentSummarizer("https://dirttrackriders.co.uk/about",openai,openai_model, explore_multiple_links=False,verbosity=True)

summary = summariser.summarize((system_prompt,user_prompt))
console = Console()
console.print(Markdown(summary))

==================== Title Page and Content Page/s ====================

(
    'Learn about Dirt Track racing in the UK. Flat Track',
    'top of page\nHOME\nABOUT\nINFO\nNEW RIDERS\nENTER A RACE\nREGULATIONS\nEUROS\nPARTNERS\nBLOG\nMore\nUse tab to
navigate through the menu items.\nABOUT\n"Good fun, fast racing, great people, increased my riding skills. Can\'t 
wait for the season to start"\n- Rick Bearcroft, DTRA Vintage Class Rider\n"The DTRA\xa0is such a welcoming club. 
I\'ve made loads of friends across the country"\n- Darren Legg, DTRA Vintage Class Rider\nDTRA FOUNDERS | Anthony 
Brown & Anna Butler\n\u200b\nCONTACT THE DTRA\nSince 2005 the UK has had its very own flat track championships. The
UK scene has been growing every year, with more new bikes and riders starting to race. Racing in the UK takes place
mainly on speedway tracks, with some use of larger horse trotting tracks and TT courses. Races are normally 
contested between 12 riders in three lines of four riders. The duration of each race is six or eight laps, with 
finals of 12 or 15 laps. Unlike in speedway, flat track bikes have a rear brake and gears.\nOur Pro class attracts 
the cream of UK Flattrack racers and often a handful of guest stars from other disciplines. Our Pro riders are of a
quality that they can compete at an international standard, and do a fantastic job representing the UK in Europe 
and the USA. The cost of DTRA club membership and race entries are kept to a minimum to encourage participation in 
the sport. Many current club riders enjoy the fact that they can race a whole season on a single rear tyre, with 
minimum additional costs.\nAll spectators and riders are welcome at our friendly race meetings, and often our races
are free to spectate. Check out the\n\'ENTER A RACE\'\npage for info on the 2025 calendar.\n© 2025 by Dirt Track 
Riders Association // See You At The Races since 2013\nbottom of page'
)

==================== User Prompt ====================

You are looking at a website titled 'Learn about Dirt Track racing in the UK. Flat Track'.

**WEBSITE CONTENT PROVIDED BELOW:**

**TASK:**
Generate a **comprehensive and structured business analysis report** based strictly on the provided website 
content.

**RESPONSE FORMAT REQUIREMENT:**
- The report **must be written in Markdown format**.
- Use **structured sections, tables (if applicable), and bullet points**.
- Ensure clarity, depth, and proper segmentation of insights.

**FINAL TOUCH:**
Provide **a summary with key takeaways**, ensuring all insights are well-organized and actionable.

**WEBSITE CONTENT STARTS BELOW:**

top of page
HOME
ABOUT
INFO
NEW RIDERS
ENTER A RACE
REGULATIONS
EUROS
PARTNERS
BLOG
More
Use tab to navigate through the menu items.
ABOUT
"Good fun, fast racing, great people, increased my riding skills. Can't wait for the season to start"
- Rick Bearcroft, DTRA Vintage Class Rider
"The DTRA is such a welcoming club. I've made loads of friends across the country"
- Darren Legg, DTRA Vintage Class Rider
DTRA FOUNDERS | Anthony Brown & Anna Butler
​
CONTACT THE DTRA
Since 2005 the UK has had its very own flat track championships. The UK scene has been growing every year, with 
more new bikes and riders starting to race. Racing in the UK takes place mainly on speedway tracks, with some use 
of larger horse trotting tracks and TT courses. Races are normally contested between 12 riders in three lines of 
four riders. The duration of each race is six or eight laps, with finals of 12 or 15 laps. Unlike in speedway, flat
track bikes have a rear brake and gears.
Our Pro class attracts the cream of UK Flattrack racers and often a handful of guest stars from other disciplines. 
Our Pro riders are of a quality that they can compete at an international standard, and do a fantastic job 
representing the UK in Europe and the USA. The cost of DTRA club membership and race entries are kept to a minimum 
to encourage participation in the sport. Many current club riders enjoy the fact that they can race a whole season 
on a single rear tyre, with minimum additional costs.
All spectators and riders are welcome at our friendly race meetings, and often our races are free to spectate. 
Check out the
'ENTER A RACE'
page for info on the 2025 calendar.
© 2025 by Dirt Track Riders Association // See You At The Races since 2013
bottom of page

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Business Analysis Report: Learn about Dirt Track Racing in the UK - Flat Track                  ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


                                                 Executive Summary                                                 

The Dirt Track Riders Association (DTRA) provides a platform for flat track motorcycle racing in the UK, focusing  
on inclusivity, competitive spirit, and community building. Since its inception in 2005, the organization has      
fostered a growing interest in flat track racing by keeping entry costs low and encouraging participation across   
various skill levels.                                                                                              

This report outlines key insights from the DTRA’s offerings and operations, exploring areas for growth, customer   
engagement strategies, and target demographics.                                                                    


                                             Core Business Information                                             

                                                    About DTRA                                                     

 • Foundation Year: 2005                                                                                           
 • Focus: Organizing flat track motorcycle races in the UK.                                                        
 • Key Attributes:                                                                                                 
    • Accessibility to participants with minimal costs associated with membership and entry.                       
    • Inclusive environment, promoting social interactions among riders and fans.                                  

                                                 Services Offered                                                  

 • Racing Events: Organized events primarily on speedway tracks and some horse trotting tracks.                    
 • Membership:                                                                                                     
    • DTRA club membership provides access to race participation.                                                  
    • Affordable costs associated with membership and race entries to enhance engagement.                          

                                                  Unique Aspects                                                   

 • Community-Oriented: Testimonials highlight friendliness and inclusiveness, enhancing the club's attractiveness. 
 • Competitive Standards: The Pro class showcases top-tier talent, enhancing the UK’s reputation in the            
   international flat track scene.                                                                                 

                                          Customer Engagement Strategies                                           

 • Free Spectating: Many races are free to watch, opening the sport to a wider audience and attracting potential   
   new riders.                                                                                                     
 • Social Events: The focus on building friendships and networks among riders promotes community growth.           


                                              Competitive Positioning                                              

 • DTRA stands out by offering a lower barrier to entry compared to traditional motorsports, with a focus on       
   community and skill development.                                                                                
 • The involvement of international riders enhances competitiveness and recognition.                               


                                     

### Multiple Page Summary

In [8]:
summariser = WebsiteContentSummarizer("https://dirttrackriders.co.uk",openai,openai_model, explore_multiple_links=True,verbosity=True)

summary = summariser.summarize((system_prompt,user_prompt),(links_filter_system_prompt,links_filter_user_prompt))
console = Console()
console.print(Markdown(summary))

==================== Weblinks Extracted ====================

[
    'https://www.dirttrackriders.co.uk/blog',
    'https://www.dirttrackriders.co.uk/enter-a-race',
    'https://www.dirttrackriders.co.uk/regulations',
    'https://www.instagram.com/dirttrackriders/?hl=en',
    'https://www.dirttrackriders.co.uk/partners',
    'https://www.dirttrackriders.co.uk/about',
    'https://www.facebook.com/DirtTrackRidersAssociationUK/',
    'https://www.dirttrackriders.co.uk/new-riders',
    'https://www.dirttrackriders.co.uk',
    'https://www.youtube.com/channel/UCLJgacR2rOImjpWiFZa4erg',
    'https://www.dirttrackriders.co.uk/info',
    'https://www.dirttrackriders.co.uk/euros'
]

==================== Relevant Links ====================

[
    {'type': 'blog', 'url': 'https://www.dirttrackriders.co.uk/blog'},
    {'type': 'about', 'url': 'https://www.dirttrackriders.co.uk/about'},
    {'type': 'partners', 'url': 'https://www.dirttrackriders.co.uk/partners'},
    {'type': 'new riders', 'url': 'https://www.dirttrackriders.co.uk/new-riders'},
    {'type': 'info', 'url': 'https://www.dirttrackriders.co.uk/info'},
    {'type': 'regulations', 'url': 'https://www.dirttrackriders.co.uk/regulations'},
    {'type': 'enter a race', 'url': 'https://www.dirttrackriders.co.uk/enter-a-race'}
]

==================== Title Page and Content Page/s ====================

(
    'Dirt Track Riders Asscoation the home of Dirt Track racing in the UK',
    [
        {
            'type': 'blog',
            'title': 'BLOG | Dirt Track Riders',
            'content': "top of page\nHOME\nABOUT\nINFO\nNEW RIDERS\nENTER A 
RACE\nREGULATIONS\nEUROS\nPARTNERS\nBLOG\nMore\nUse tab to navigate through the menu items.\nAll Posts\nRace 
Previews\nReports\nSearch\nLog in / Sign up\nalexraby\nApr 16, 2023\n3 min read\n2023 DTRA RD1 GREENFIELD OVAL 
[REPORT]\nBirtwistle makes successful start to title defence Reigning DTRA UK Flat Track National Champion Gary 
Birtwistle opened the 2023 season...\n258 views\n0 comments\nPost not marked as liked\nusefultone\nSep 7, 2021\n4 
min read\nRace Preview: DTRA Nationals Round 6 & Hooligans Round 4 2021, Greenfield Dirt Track TT\nIt’s hard to 
believe that this year’s DTRA flat track racing season is almost over, but it’s time to get ready for our sixth and
final...\n807 views\n0 comments\nPost not marked as liked\nalexraby\nAug 17, 2021\n3 min read\n2021 DTRA RD4 KING'S
LYNN [REPORT]\nHALES TAKES COMMAND Toby Hales took a major step towards recovering the Indian Motorcycle DTRA UK 
Flat Track Nationals title he last won...\n215 views\n0 comments\nPost not marked as liked\nusefultone\nAug 8, 
2021\n3 min read\nRace Preview: DTRA Nationals Round 4 & Hooligans Round 3 2021, Adrian Flux Arena, King’s 
Lynn\nThe DTRA is heading back to Adrian Flux Arena in King’s Lynn, Norfolk for the next round of flat track racing
on Sunday 15th August.\n249 views\n0 comments\nPost not marked as liked\n© 2025 by Dirt Track Riders Association //
See You At The Races since 2013\nbottom of page"
        },
        {
            'type': 'about',
            'title': 'Learn about Dirt Track racing in the UK. Flat Track',
            'content': 'top of page\nHOME\nABOUT\nINFO\nNEW RIDERS\nENTER A 
RACE\nREGULATIONS\nEUROS\nPARTNERS\nBLOG\nMore\nUse tab to navigate through the menu items.\nABOUT\n"Good fun, fast
racing, great people, increased my riding skills. Can\'t wait for the season to start"\n- Rick Bearcroft, DTRA 
Vintage Class Rider\n"The DTRA\xa0is such a welcoming club. I\'ve made loads of friends across the country"\n- 
Darren Legg, DTRA Vintage Class Rider\nDTRA FOUNDERS | Anthony Brown & Anna Butler\n\u200b\nCONTACT THE DTRA\nSince
2005 the UK has had its very own flat track championships. The UK scene has been growing every year, with more new 
bikes and riders starting to race. Racing in the UK takes place mainly on speedway tracks, with some use of larger 
horse trotting tracks and TT courses. Races are normally contested between 12 riders in three lines of four riders.
The duration of each race is six or eight laps, with finals of 12 or 15 laps. Unlike in speedway, flat track bikes 
have a rear brake and gears.\nOur Pro class attracts the cream of UK Flattrack racers and often a handful of guest 
stars from other disciplines. Our Pro riders are of a quality that they can compete at an international standard, 
and do a fantastic job representing the UK in Europe and the USA. The cost of DTRA club membership and race entries
are kept to a minimum to encourage participation in the sport. Many current club riders enjoy the fact that they 
can race a whole season on a single rear tyre, with minimum additional costs.\nAll spectators and riders are 
welcome at our friendly race meetings, and often our races are free to spectate. Check out the\n\'ENTER A 
RACE\'\npage for info on the 2025 calendar.\n© 2025 by Dirt Track Riders Association // See You At The Races since 
2013\nbottom of page'
        },
        {
            'type': 'partners',
            'title': 'PARTNERS | Dirt Track Riders',
            'content': 'top of page\nHOME\nABOUT\nINFO\nNEW RIDERS\nENTER A 
RACE\nREGULATIONS\nEUROS\nPARTNERS\nBLOG\nMore\nUse tab to navigate through the menu items.\nPARTNERS\nBECOME A 
DTRA PARTNER\nThe DTRA are proud to work with\xa0many likeminded brands and companies who\xa0colaberate wit

==================== User Prompt ====================

You are looking at a website titled 'Dirt Track Riders Asscoation the home of Dirt Track racing in the UK'.

**WEBSITE CONTENT PROVIDED BELOW:**

**TASK:**
Generate a **comprehensive and structured business analysis report** based strictly on the provided website 
content.

**RESPONSE FORMAT REQUIREMENT:**
- The report **must be written in Markdown format**.
- Use **structured sections, tables (if applicable), and bullet points**.
- Ensure clarity, depth, and proper segmentation of insights.

**FINAL TOUCH:**
Provide **a summary with key takeaways**, ensuring all insights are well-organized and actionable.

**WEBSITE CONTENT STARTS BELOW:**

[{'type': 'blog', 'title': 'BLOG | Dirt Track Riders', 'content': "top of page\nHOME\nABOUT\nINFO\nNEW 
RIDERS\nENTER A RACE\nREGULATIONS\nEUROS\nPARTNERS\nBLOG\nMore\nUse tab to navigate through the menu items.\nAll 
Posts\nRace Previews\nReports\nSearch\nLog in / Sign up\nalexraby\nApr 16, 2023\n3 min read\n2023 DTRA RD1 
GREENFIELD OVAL [REPORT]\nBirtwistle makes successful start to title defence Reigning DTRA UK Flat Track National 
Champion Gary Birtwistle opened the 2023 season...\n258 views\n0 comments\nPost not marked as 
liked\nusefultone\nSep 7, 2021\n4 min read\nRace Preview: DTRA Nationals Round 6 & Hooligans Round 4 2021, 
Greenfield Dirt Track TT\nIt’s hard to believe that this year’s DTRA flat track racing season is almost over, but 
it’s time to get ready for our sixth and final...\n807 views\n0 comments\nPost not marked as liked\nalexraby\nAug 
17, 2021\n3 min read\n2021 DTRA RD4 KING'S LYNN [REPORT]\nHALES TAKES COMMAND Toby Hales took a major step towards 
recovering the Indian Motorcycle DTRA UK Flat Track Nationals title he last won...\n215 views\n0 comments\nPost not
marked as liked\nusefultone\nAug 8, 2021\n3 min read\nRace Preview: DTRA Nationals Round 4 & Hooligans Round 3 
2021, Adrian Flux Arena, King’s Lynn\nThe DTRA is heading back to Adrian Flux Arena in King’s Lynn, Norfolk for the
next round of flat track racing on Sunday 15th August.\n249 views\n0 comments\nPost not marked as liked\n© 2025 by 
Dirt Track Riders Association // See You At The Races since 2013\nbottom of page"}, {'type': 'about', 'title': 
'Learn about Dirt Track racing in the UK. Flat Track', 'content': 'top of page\nHOME\nABOUT\nINFO\nNEW 
RIDERS\nENTER A RACE\nREGULATIONS\nEUROS\nPARTNERS\nBLOG\nMore\nUse tab to navigate through the menu 
items.\nABOUT\n"Good fun, fast racing, great people, increased my riding skills. Can\'t wait for the season to 
start"\n- Rick Bearcroft, DTRA Vintage Class Rider\n"The DTRA\xa0is such a welcoming club. I\'ve made loads of 
friends across the country"\n- Darren Legg, DTRA Vintage Class Rider\nDTRA FOUNDERS | Anthony Brown & Anna 
Butler\n\u200b\nCONTACT THE DTRA\nSince 2005 the UK has had its very own flat track championships. The UK scene has
been growing every year, with more new bikes and riders starting to race. Racing in the UK takes place mainly on 
speedway tracks, with some use of larger horse trotting tracks and TT courses. Races are normally contested between
12 riders in three lines of four riders. The duration of each race is six or eight laps, with finals of 12 or 15 
laps. Unlike in speedway, flat track bikes have a rear brake and gears.\nOur Pro class attracts the cream of UK 
Flattrack racers and often a handful of guest stars from other disciplines. Our Pro riders are of a quality that 
they can compete at an international standard, and do a fantastic job representing the UK in Europe and the USA. 
The cost of DTRA club membership and race entries are kept to a minimum to encourage participation in the sport. 
Many current club riders enjoy the fact that they can race a whole season on a single rear tyre, with minimum 
additional costs.\nAll spectators and riders are welcome at our friendly race meetings, and often our races are 
free to spectate. Check out the\n\'ENTER A RACE\'\npage for info on the 2025 calendar.\n© 2025 by Dirt Track Ri

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                         Business Analysis Report: Dirt Track Riders Association (DTRA)                          ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


                                                 Executive Summary                                                 

The Dirt Track Riders Association (DTRA) is a prominent organization focusing on flat track motorcycle racing in   
the UK. Since its inception in 2005, the DTRA has significantly contributed to the growth of the sport, creating a 
welcoming environment that encourages participation across various demographics. The club not only hosts national  
championships but also engages in community-building activities that foster relationships among riders.            


                                             Core Business Information                                             

                                                Mission and Vision                                                 

 • Mission: To promote and nurture the flat track motorcycle racing community in the UK by providing engaging and  
   accessible racing events.                                                                                       
 • Vision: To be the leading organization for flat track racing, where enthusiasts experience “good fun, fast      
   racing, and great people.”                                                                                      

                                              Services and Offerings                                               

 • Racing Events: Organizes a structured racing calendar including national championships, hooligan races, and     
   events hosted at speedway and trotting tracks.                                                                  
 • New Rider Support: Provides extensive resources and guidance for newcomers, including information about classes,
   regulations, and educational racing schools.                                                                    
 • Community Engagement: Encourages social interaction among members and supports networking through race meetings 
   and events.                                                                                                     

                                                 Pricing Structure                                                 

 • Membership Fees:                                                                                                
    • Adult Membership: £40                                                                                        
    • Youth Membership: £30                                                                                        
 • Race Entry Fees:                                                                                                
    • One Class: £50                                                                                               
    • Two Classes: £70                                                                                             
    • Youth Entry: £40                                                                                             

                                                  Unique Aspects                                                   

 • Affordability: The DTRA keeps costs low to enhance accessibility, allowing more participants to enjoy the sport.
 • Diverse Classes: Ranges from youth categories to professional levels, ensuring inclusivity for various age      
   groups and skill levels.                                                                                        
 • Corporate Partnerships: Collaborates with brands like Royal Enfield, which helps enhance the profile and reach  
   of the Association.                     